In [1]:
import cvxpy as cp
import numpy as np

In [2]:
def indexed_constraints(x, b):
    """Returns vectorized constraints: x <= b (elementwise)"""
    return [x <= b]  # Elementwise broadcasting

# Problem dimensions
rows, cols = 5, 4

# Define decision variables (2D matrix)
x = cp.Variable((rows, cols))

# Define bounds as a NumPy array
b = np.array([[i + j + 1 for j in range(cols)] for i in range(rows)])

# Get constraints from function (vectorized)
constraints = indexed_constraints(x, b)

# Define an objective function (example: minimize sum of x)
objective = cp.Maximize(cp.sum(x))

# Define the problem
prob = cp.Problem(objective, constraints)

# Solve the problem
prob.solve()

# Print results
print("Optimal x:\n", x.value)

Optimal x:
 [[1. 2. 3. 4.]
 [2. 3. 4. 5.]
 [3. 4. 5. 6.]
 [4. 5. 6. 7.]
 [5. 6. 7. 8.]]


In [3]:
def indexed_constraints(x, b):
    """Returns a list of constraints: x[i, j] <= b[i, j] for all i, j"""
    rows, cols = x.shape
    return [x[i, j] <= b[i, j] for i in range(rows) for j in range(cols)]

# Problem dimensions
rows, cols = 5, 4

# Define decision variables (2D matrix)
x = cp.Variable((rows, cols))

# Define bounds as a NumPy array
b = np.array([[i + j + 1 for j in range(cols)] for i in range(rows)])

# Get constraints from function
constraints = indexed_constraints(x, b)

# Define an objective function (example: minimize sum of x)
objective = cp.Maximize(cp.sum(x))

# Define the problem
prob = cp.Problem(objective, constraints)

# Solve the problem
prob.solve()

# Print results
print("Optimal x:\n", x.value)

Optimal x:
 [[1. 2. 3. 4.]
 [2. 3. 4. 5.]
 [3. 4. 5. 6.]
 [4. 5. 6. 7.]
 [5. 6. 7. 8.]]


In [9]:
# Import packages.
import cvxpy as cp
import numpy as np

# Generate a random feasible SOCP.
m = 3
n = 10
p = 5
n_i = 5
np.random.seed(2)
f = np.random.randn(n)
A = []
b = []
c = []
d = []
x0 = np.random.randn(n)
for i in range(m):
    A.append(np.random.randn(n_i, n))
    b.append(np.random.randn(n_i))
    c.append(np.random.randn(n))
    d.append(np.linalg.norm(A[i] @ x0 + b, 2) - c[i].T @ x0)
F = np.random.randn(p, n)
g = F @ x0

# Define and solve the CVXPY problem.
x = cp.Variable(n)
# We use cp.SOC(t, x) to create the SOC constraint ||x||_2 <= t.
soc_constraints = [
    cp.SOC(c[i].T @ x + d[i], A[i] @ x + b[i]) for i in range(m)
]
prob = cp.Problem(
    cp.Maximize(x.sum()),
    soc_constraints + [F @ x == g])

prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution x is")
print(x.value)
for i in range(m):
    print("SOC constraint %i dual variable solution" % i)
    print(soc_constraints[i].dual_value)


The optimal value is 9.342278754284528
A solution x is
[ 1.40846102  2.40182762  1.71873406  0.28552901  1.46266482 -1.361357
  0.12489988  1.65187869  0.79166118  0.85797947]
SOC constraint 0 dual variable solution
[array([0.38116578]), array([[ 0.24283811],
       [ 0.02199461],
       [-0.02135339],
       [ 0.08164417],
       [ 0.2805557 ]])]
SOC constraint 1 dual variable solution
[array([0.38115783]), array([[-0.15697698],
       [ 0.21316597],
       [-0.24500629],
       [ 0.10853708],
       [ 0.05823578]])]
SOC constraint 2 dual variable solution
[array([0.85344598]), array([[ 0.0472224 ],
       [-0.48055749],
       [ 0.20294042],
       [-0.65740584],
       [ 0.14777455]])]


In [69]:
m = 10
p = np.random.random(m)
q = np.random.random(m)
v = np.random.random(m)

f_res = (np.square(p) + np.square(q))/ v

f = cp.Variable(m)
soc_constraints = [
    cp.SOC(
        (v[i] + f[i]),
        cp.hstack([
            2* p[i], 2* q[i], f[i] - v[i]
        ])
    )
    for i in range(m)
]
prob = cp.Problem(
    cp.Minimize(0.001*f.sum()),
    soc_constraints)
prob.solve()


np.abs(f_res - f.value).max()

2.6748214315558405e-06

array([3.28688055e-08, 2.23123222e-08, 1.03346466e-08, 1.48692814e-08,
       2.30056711e-08, 4.65656964e-08, 1.51717469e-09, 4.86954921e-09,
       2.91072138e-08, 1.89386002e-08])